Transfer Learning Keras Implementation: Chimp Call Classification

In [1]:
#Import Dependencies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os


In [2]:
#Parameters for Our Chimp Call Classifier
NUM_CLASSES = 11
CHANNELS = 3
IMAGE_RESIZE = 128
VGGISH_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

LOSS_METRICS = ['accuracy']

NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100

BATCH_SIZE_TESTING = 1

VGGISH_CHECKPOINT_PATH = "./vggish/model_parameters/vggish_weights.ckpt"

In [18]:
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Flatten, Input, InputLayer
from vggish import vggish_keras

In [19]:
#Preprocess audio and labels

In [20]:
#Run this at a later point in time after getting vggish right
feature_model = vggish_keras.get_vggish_keras()
feature_model.load_weights(VGGISH_CHECKPOINT_PATH)
for layer in feature_model.layers:
    layer.trainable = False
feature_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 96, 64, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 96, 64, 64)        640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 48, 32, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 48, 32, 128)       73856     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 24, 16, 128)       0         
_________________________________________________________________
conv3_1 (Conv2D)             (None, 24, 16, 256)       295168    
_________________________________________________________________
conv3_2 (Conv2D)             (None, 24, 16, 256)       590080    
__________

In [21]:
feature_model.layers

In [24]:
F = feature_model.output
#F=MaxPooling2D()(F)
F=Dense(128, input_shape = (128,),activation='relu')(F) #we add dense layers so that the model can learn more complex functions and classify for better results.
preds=Dense(11,activation='softmax')(F) #final layer with softmax activation
full_model = Model(feature_model.input, preds)
full_model.summary()

AttributeError: 'Dense' object has no attribute 'outbound_nodes'

In [16]:

top_model = Sequential()
#top_model.add(feature_model)
top_model.add(Dense(128,input_shape=feature_model.output_shape[1:]))
top_model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))
top_model.summary()
#top_model = feature_model
# top_model.add(feature_model)
# top_model = Dense((None,  128), activation = 'relu')(feature_model)
# top_model = Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION)(top_model)
# top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 11)                1419      
Total params: 17,931
Trainable params: 17,931
Non-trainable params: 0
_________________________________________________________________


In [17]:
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

LOSS_METRICS = ['accuracy']

#sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
top_model.compile(optimizer = Adam, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

ValueError: Could not interpret optimizer identifier: <class 'keras.optimizers.Adam'>